## Zero Bond

In [108]:
import pytest
import tensorflow as tf
import pandas as pd
import numpy as np
from scripts.utils.utils.utils import (
    ZeroBond,
    FinanceUtils
)
from scripts.trainer.trainer import simulate

T = 8

T_dicts = {
    t: 0 for t in range(1, T)
}
results_t = {
    t: None for t in range(1, T)
}
# Steps per year
nsteps_per_year = 48
# Params for check
for t in range(1, T):
    nsteps = t * nsteps_per_year
    # Fixed params
    simple_params = {
        "T": t,
        "N_steps": nsteps,
        "dim": 1,
        "sigma": 0.01,
        "nsims": 1000
    }
    data, _ = simulate(
        T = simple_params["T"], 
        N_steps = simple_params["N_steps"], 
        dim = simple_params["dim"],
        sigma = simple_params["sigma"], 
        nsims = simple_params["nsims"]
    )

    # Get data to process the ZeroBound
    delta_x = data.delta_x_0.values
    xt = data.X_0.values
    dt = data.dt.values
    t_unique = data.dt.unique()
    dict_C = {dt:FinanceUtils.C(dt, sigma_value = 0.01) for dt in t_unique}
    ct = data.apply(lambda x: dict_C[x['dt']], axis = 1)
    nt = ZeroBond.N_tensor(dt,xt,ct)
    # Convert to tensors
    xt = tf.convert_to_tensor(xt, dtype = tf.float64)
    delta_x = tf.convert_to_tensor(delta_x,dtype = tf.float64)
    dt = tf.convert_to_tensor(dt, dtype = tf.float64)
    ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
    T = tf.Variable(np.float64(T), name = 'T', trainable=False)
    # Fix the batch size
    batch_size = int(xt.shape[0] / nsteps)
    # Real values
    v_real = ZeroBond.Z_tensor(xt, dt, T, ct)
    v_real_reshaped = tf.reshape(v_real,(batch_size,nsteps))
    n_tensor = ZeroBond.N_tensor(dt, xt, ct)
    # Derivative:
    xt = tf.Variable(xt, name = 'xn', trainable = True)
    dt = tf.Variable(dt, name = 'tn', trainable = False)
    ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
    with tf.GradientTape() as tape:
        y = ZeroBond.Z_normalized(xt, dt, T, ct)
    grad_df = tape.gradient(y, {'xn':xt})
    grads = grad_df['xn']
    # Simulate - LGM step:
    grads_reshaped = tf.reshape(grads, (batch_size, nsteps))
    xt_reshaped = tf.reshape(xt, (batch_size, nsteps))
    delta_x_reshaped = tf.reshape(delta_x, (batch_size, nsteps))
    # Calculate the MVP
    v = np.ones((batch_size, nsteps)) * np.float64(v_real_reshaped[0, 0])
    for i in range(1, nsteps):
        v[:, i] = (v[:, i - 1] + grads_reshaped[:, i - 1] * delta_x_reshaped[:, i])
    # Calculate errors absolute
    v_real = np.array(tf.reshape(v_real_reshaped, -1))
    v_column = np.array(tf.reshape(v, -1)) * n_tensor.numpy()
    dt_list = np.array(dt)
    df_results = pd.DataFrame(zip(xt.numpy(), v_real, v_column, dt_list, n_tensor.numpy()), columns = ["xt", "v_real","v_est","dt", "n"])
    # Error
    df_results["absolute_error"] = (df_results.v_real - df_results.v_est).abs()
    # Store results
    results_t[t] = df_results.absolute_error.mean()

#### Aggregated result

In [109]:
results_t

{1: 0.00023597587109969338,
 2: 0.0003386900836775168,
 3: 0.0004269469030297706,
 4: 0.0005199961112154946,
 5: 0.0006344051798412212,
 6: 0.0008010608678608267,
 7: 0.0010065686122013705}

## IRS

In [ ]:
import pytest
import tensorflow as tf
import pandas as pd
import numpy as np
from scripts.utils.utils.utils import (
    IRS,
    ZeroBond,
    FinanceUtils
)
from scripts.trainer.trainer import simulate

T = 8

T_dicts = {
    t: 0 for t in range(1, T)
}
results_t = {
    t: None for t in range(1, T)
}
# Steps per year
nsteps_per_year = 48
# Params for check
for t in range(1, T):
    nsteps = t * nsteps_per_year
    # Fixed params
    simple_params = {
        "T": t,
        "N_steps": nsteps,
        "dim": 1,
        "sigma": 0.01,
        "nsims": 1000
    }
    data, _ = simulate(
        T = simple_params["T"], 
        N_steps = simple_params["N_steps"], 
        dim = simple_params["dim"],
        sigma = simple_params["sigma"], 
        nsims = simple_params["nsims"]
    )

    # Get data to process the ZeroBound
    delta_x = data.delta_x_0.values
    xt = data.X_0.values
    dt = data.dt.values
    t_unique = data.dt.unique()
    dict_C = {dt:FinanceUtils.C(dt, sigma_value = 0.01) for dt in t_unique}
    ct = data.apply(lambda x: dict_C[x['dt']], axis = 1)
    nt = ZeroBond.N_tensor(dt,xt,ct)
    # Convert to tensors
    xt = tf.convert_to_tensor(xt, dtype = tf.float64)
    delta_x = tf.convert_to_tensor(delta_x,dtype = tf.float64)
    dt = tf.convert_to_tensor(dt, dtype = tf.float64)
    ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
    T = tf.Variable(np.float64(T), name = 'T', trainable=False)
    # Fix the batch size
    batch_size = int(xt.shape[0] / nsteps)
    # Real values
    v_real = IRS.I(xt, dt, T, ct)
    v_real_reshaped = tf.reshape(v_real,(batch_size,nsteps))
    n_tensor = ZeroBond.N_tensor(dt, xt, ct)
    # Derivative:
    xt = tf.Variable(xt, name = 'xn', trainable = True)
    dt = tf.Variable(dt, name = 'tn', trainable = False)
    ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
    with tf.GradientTape() as tape:
        y = IRS(xt, dt, T, ct)
    grad_df = tape.gradient(y, {'xn':xt})
    grads = grad_df['xn']
    # Simulate - LGM step:
    grads_reshaped = tf.reshape(grads, (batch_size, nsteps))
    xt_reshaped = tf.reshape(xt, (batch_size, nsteps))
    delta_x_reshaped = tf.reshape(delta_x, (batch_size, nsteps))
    # Calculate the MVP
    v = np.ones((batch_size, nsteps)) * np.float64(v_real_reshaped[0, 0])
    for i in range(1, nsteps):
        v[:, i] = (v[:, i - 1] + grads_reshaped[:, i - 1] * delta_x_reshaped[:, i])
    # Calculate errors absolute
    v_real = np.array(tf.reshape(v_real_reshaped, -1))
    v_column = np.array(tf.reshape(v, -1)) * n_tensor.numpy()
    dt_list = np.array(dt)
    df_results = pd.DataFrame(zip(xt.numpy(), v_real, v_column, dt_list, n_tensor.numpy()), columns = ["xt", "v_real","v_est","dt", "n"])
    # Error
    df_results["absolute_error"] = (df_results.v_real - df_results.v_est).abs()
    # Store results
    results_t[t] = df_results.absolute_error.mean()